In [ ]:
# ------------------------------------------------------------------------
#
# TITLE - plot_fit_mock.ipynb
# AUTHOR - James Lane
# PROJECT - ges-mass
#
# ------------------------------------------------------------------------
#
# Docstrings and metadata:
'''Fit the mock density profiles
'''

__author__ = "James Lane"

In [ ]:
### Imports

## Basic
import numpy as np
import sys, os, pdb, copy, time, multiprocessing, warnings, dill as pickle

## Matplotlib
import matplotlib
from matplotlib import pyplot as plt
import corner

## Astropy
from astropy import units as apu
from astropy.coordinates import SkyCoord

## galpy
from galpy import orbit
from galpy import potential
from galpy import actionAngle as aA
from galpy import df

## Analysis
import mwdust
import emcee
import scipy.optimize
from isodist import Z2FEH,FEH2Z

## Project-specific
sys.path.insert(0,'../../src/')
from ges_mass import mass as pmass
from ges_mass import densprofiles as pdens
from ges_mass import util as putil
from ges_mass import plot as pplot

### Notebook setup

%matplotlib inline
plt.style.use('../../src/mpl/project.mplstyle') # This must be exactly here
%config InlineBackend.figure_format = 'retina'
%load_ext autoreload
%autoreload 2

### Keywords, Pathing, Loading, Data Preparation

In [ ]:
# %load ../../../src/nb_modules/keywords_pathing_loading_data_prep.py
## Keywords
cdict = putil.load_config_to_dict()
keywords = ['BASE_DIR','APOGEE_DR','APOGEE_RESULTS_VERS','GAIA_DR','NDMOD',
            'DMOD_MIN','DMOD_MAX','LOGG_MIN','LOGG_MAX','FEH_MIN','FEH_MAX',
            'FEH_MIN_GSE','FEH_MAX_GSE','DF_VERSION','KSF_VERSION','NPROCS',
            'RO','VO','ZO']
base_dir,apogee_dr,apogee_results_vers,gaia_dr,ndmod,dmod_min,dmod_max,\
    logg_min,logg_max,feh_min,feh_max,feh_min_gse,feh_max_gse,df_version,\
    ksf_version,nprocs,ro,vo,zo = putil.parse_config_dict(cdict,keywords)
logg_range = [logg_min,logg_max]
feh_range = [feh_min,feh_max]
feh_range_gse = [feh_min_gse,feh_max_gse]
feh_range_all = [feh_min,feh_max_gse]

## Pathing
fit_paths = putil.prepare_paths(base_dir,apogee_dr,apogee_results_vers,gaia_dr,
                                df_version,ksf_version)
data_dir,version_dir,ga_dir,gap_dir,df_dir,ksf_dir,fit_dir = fit_paths

## Filenames
fit_filenames = putil.prepare_filenames(ga_dir,gap_dir,feh_range_gse)
apogee_SF_filename,apogee_effSF_filename,apogee_effSF_mask_filename,\
    iso_grid_filename,clean_kinematics_filename = fit_filenames

## File loading and data preparation
fit_stuff,other_stuff = putil.prepare_fitting(fit_filenames,
    [ndmod,dmod_min,dmod_max], ro,zo,return_other=True)
apogee_effSF_mask,dmap,iso_grid,jkmins,dmods,ds,effsel_grid,apof,\
    allstar_nomask,orbs_nomask = fit_stuff
Rgrid,phigrid,zgrid = effsel_grid

## Load the distribution functions
df_filename = df_dir+'dfs.pkl'
betas = [0.3,0.8]
dfs = putil.load_distribution_functions(df_filename, betas)

## Figures
fig_dir = './fig/mock/'
os.makedirs(fig_dir,exist_ok=True)

### Global parameters

In [ ]:
# %load ../../../src/nb_modules/global_fitting_params.py
## general kwargs
verbose = True

## HaloFit kwargs (ordering follows HaloFit.__init__)
# allstar and orbs loaded in prep cell
init = None
init_type = 'ML'
# fit_type provided at runtime
mask_disk = True
mask_halo = True
# densfunc, selec provided at runtime
# effsel, effsel_grid, effsel_mask, dmods loaded in prep cell
nwalkers = 100
nit = int(1e6)
ncut = int(1e3)
# usr_log_prior provided at runtime
n_mass = 2000 # int(nwalkers*(nit-ncut))
int_r_range = [2.,70.]
iso = None # Will read from iso_grid_filename
# iso_filename, jkmins loaded in prep cell
# feh_range provided at runtime
# logg_range loaded in config cell
# fit_dir, gap_dir, ksf_dir loaded in prep cell
# version provided at runtime
# ro, vo, zo loaded in config cell

hf_kwargs = {## HaloFit parameters
             'allstar':allstar_nomask,
             'orbs':orbs_nomask,
             'init':init,
             'init_type':init_type,
             # 'fit_type':fit_type, # provided at runtime
             'mask_disk':mask_disk,
             'mask_halo':mask_halo,
             ## _HaloFit parameters
             # 'densfunc':densfunc, # provided at runtime
             # 'selec':selec, # provided at runtime
             'effsel':apof,
             'effsel_mask':apogee_effSF_mask,
             'effsel_grid':effsel_grid,
             'dmods':dmods,
             'nwalkers':nwalkers,
             'nit':nit,
             'ncut':ncut,
             # 'usr_log_prior':usr_log_prior, # provided at runtime
             'n_mass':n_mass,
             'int_r_range':int_r_range,
             'iso':iso,
             'iso_filename':iso_grid_filename,
             'jkmins':jkmins,
             # 'feh_range':feh_range, # provided at runtime
             'logg_range':logg_range,
             'fit_dir':fit_dir,
             'gap_dir':gap_dir,
             'ksf_dir':ksf_dir,
             # 'version':version, # provided at runtime
             'verbose':verbose,
             'ro':ro,
             'vo':vo,
             'zo':zo}

## pmass.fit() function kwargs
# nprocs set in config file
force_fit = True
mle_init = True
just_mle = False
return_walkers = True
optimizer_method = 'Powell'
mass_int_type = 'spherical_grid'
batch_masses = True
make_ml_aic_bic = True
calculate_masses = True
post_optimization = True
mcmc_diagnostic = True

fit_kwargs = {# 'nprocs':nprocs, # Normally given at runtime 
              'force_fit':force_fit,
              'mle_init':mle_init,
              'just_mle':just_mle,
              'return_walkers':return_walkers,
              'optimizer_method':optimizer_method,
              'mass_int_type':mass_int_type,
              'batch_masses':batch_masses,
              'make_ml_aic_bic':make_ml_aic_bic,
              'calculate_masses':calculate_masses,
              'post_optimization':post_optimization,
              'mcmc_diagnostic':mcmc_diagnostic,
              }

### Convenience functions

Plots to make:
- Each needs its own kwargs
- Each needs to be called flexibly

1. Corner plot
- With / without masses
- Needs kwargs


2. Just masses
- Needs kwargs


3. Distance modulus posterior
- Needs kwargs
- Include log(radius on a twin axis)


4. Density profile / contours
- Needs kwargs
- Needs a hole in the center of the figure


5. Density vs. effective radius
- Should include effective volume to show where 

In [ ]:
def make_plots(hf, make_arr, corner_kwargs={}, masses_kwargs={}, 
               dmod_posterior_kwargs={}, density_slice_kwargs={}, 
               density_re_kwargs={}, return_figs=True):
    '''make_arr is corner,masses,dmod_posterior,density_slice,density_re
    '''
    
    # Unpack boolean arrays
    make_corner, make_masses, make_dmod_posterior, make_density_slice,\
        make_density_re = make_arr
    
    # Make sure that the HaloFit object has the results loaded
    hf.get_results()
    hf.get_loglike_ml_aic_bic()
    
    figs = []
    
    # Make each figure
    if make_corner:
        print('Making corner figure')
        fig = pplot.plot_corner(hf, **corner_kwargs)
        fig.savefig(hf.fit_fig_dir+'corner.png')
        figs.append(fig)
        
    if make_masses:
        print('Making masses figure')
        fig = pplot.plot_masses(hf, **masses_kwargs)
        fig.savefig(hf.fit_fig_dir+'masses.png')
        figs.append(fig)
        
    if make_dmod_posterior:
        print('Making distance modulus posterior figure')
        fig = pplot.plot_distmod_posterior(hf, **dmod_posterior_kwargs)
        fig.savefig(hf.fit_fig_dir+'dmod_posterior.png')
        figs.append(fig)
        
    if make_density_slice:
        print('Making density slice figure')
        fig = pplot.plot_density_xyz(hf, **density_slice_kwargs)
        fig.savefig(hf.fit_fig_dir+'density_slice.png')
        figs.append(fig)
        
    if make_density_re:
        print('Make density vs effective radius figure')
        fig = pplot.plot_density_re(hf, **density_re_kwargs)
        fig.savefig(hf.fit_fig_dir+'density_re.png')
        figs.append(fig)
    
    if return_figs:
        return figs

### Some other preparation for mocks

In [ ]:
# Make the isochrone grid into the specific single mock isochrone
iso_z = 0.0010 # Normal parameters for single isochrone
iso_log_age = 10.0
iso = iso_grid[(iso_grid['Zini']==iso_z) & (iso_grid['logAge']==iso_log_age)]

# Potential and action angle Staeckel object
mwpot = potential.MWPotential2014
potential.turn_physical_on(mwpot,ro=ro,vo=vo)
phi0 = potential.evaluatePotentials(mwpot,1e10,0).value
aAS = aA.actionAngleStaeckel(pot=mwpot, delta=0.4, c=True, ro=ro, vo=vo)

In [ ]:
# Halo mock information
halo_mock_number = '79'
halo_mock_path = '../apo_mocks/data/mock_'+halo_mock_number+'/'
halo_mock_densfunc = pdens.triaxial_single_angle_zvecpa
halo_mock_mass = 1e8
r_range_halo = [2.,55.]
halo_mock_alpha = 3.5
# halo_mock_rc = 25
halo_mock_p = 0.8
halo_mock_q = 0.5
halo_mock_eta = 1./np.sqrt(2.)
halo_mock_theta = np.pi/2.
halo_mock_zvec = pdens.eta_theta_to_zvec(halo_mock_eta,halo_mock_theta)
halo_mock_pa = np.pi/5.
# halo_mock_eta,halo_mock_theta = pdens.zvec_to_eta_theta(halo_mock_zvec)
halo_mock_params = [halo_mock_alpha,#halo_mock_rc,
                    halo_mock_p,halo_mock_q,
                    halo_mock_theta,halo_mock_eta,halo_mock_pa]
halo_mock_params_norm = pdens.normalize_parameters(halo_mock_params,
        halo_mock_densfunc)

# Disk mock information
disk_mock_number = '70'
disk_mock_path = '../apo_mocks/data/mock_'+disk_mock_number+'/'
disk_mock_densfunc = pdens.exp_disk
disk_mock_mass = 2e8
disk_mock_params = [1/2., 1/0.8]
R_range_disk = [2.,55.]
z_max_disk = 16.

# DF mixture information
mixture_arr = np.array([0.5,0.5])

# putil.kinematic_selection_mask keywords
kinematic_selection_mask_kwargs = {
        'space':'eLz',
        'selection':None, # Just the dict version of space, alternate
        'selection_version':'current'}

# HaloFit kwargs for the mock
hf_kwargs_mock = copy.deepcopy(hf_kwargs)
hf_kwargs_mock['init_type'] = 'truths'
hf_kwargs_mock['verbose'] = verbose
hf_kwargs_mock['feh_range'] = None
hf_kwargs_mock['iso_feh'] = Z2FEH(iso_z)
hf_kwargs_mock['iso_age'] = iso_log_age
hf_kwargs_mock['iso'] = iso
hf_kwargs_mock['iso_filename'] = None

# kwargs for mass_from_density_samples
mass_from_density_samples_kwargs = {}

# fdisk calculation keywords
fdisk_targ = 0.4
mass_from_density_samples_kwargs = {}
fdisk_calc_kwargs = {
        'fdisk_targ':fdisk_targ,
        'halo_densfunc':halo_mock_densfunc,
        'halo_params':halo_mock_params,
        'halo_mass':halo_mock_mass,
        'disk_densfunc':disk_mock_densfunc,
        'disk_params':disk_mock_params,
        'disk_mass':disk_mock_mass,
        'r_range_halo':r_range_halo,
        'R_range_disk':R_range_disk,
        'z_max_disk':z_max_disk,
        'ro': ro, 'zo': zo,
        'mass_from_density_samples_kwargs':mass_from_density_samples_kwargs
        }

### Plot results of the fits to the mocks

In [ ]:
make_arr = [True,False,True,False,False]
_range_fac = 0.99
corner_range = [_range_fac,_range_fac,_range_fac,
                (0.-0.1,2*np.pi+0.1),(0.75,1.01),
                _range_fac]

corner_kwargs={'samples':None,
               'plot_mass':True,
               'thin':None,
               'thin_to':int(1e5),
               'quantiles':[0.16,0.5,0.84],
               'show_titles':True,
               'truths':'mock_truths',
               'corner_kwargs':{'truth_color':'Red',
                                #'range':corner_range,
                               }}

masses_kwargs={'mass_in_log':True,
               'truths':'mock_truths',
               'corner_kwargs':{'truth_color':'Red',
                               }}

dmod_posterior_kwargs={'nrand':50,
                       'hist_kwargs':{'bins':20,
                                      'color':'Black'
                                     },
                       'lines_kwargs':{'color':'DodgerBlue',
                                       'alpha':0.2
                                      },
                      }

density_slice_kwargs={}

density_re_kwargs={}

#### First just do a mock, no KSF, no disk

In [ ]:
fit_type = 'mock'
mock_truths = copy.deepcopy(halo_mock_params) # Needs to be normalized parameters
_hf_kwargs_mock = copy.deepcopy(hf_kwargs_mock)
_hf_kwargs_mock['densfunc'] = pdens.triaxial_single_angle_zvecpa
_hf_kwargs_mock['fit_type'] = fit_type
_hf_kwargs_mock['selec'] = None
_hf_kwargs_mock['version'] = 'hf_mock_halo_'+halo_mock_number
_hf_kwargs_mock['truths'] = mock_truths
_hf_kwargs_mock['truth_mass'] = halo_mock_mass
_hf_kwargs_mock['truths_normed'] = False
_hf_kwargs_mock['int_r_range'] = r_range_halo

hf = pmass.mock_to_hf(
    mock_path= halo_mock_path,
    dfs= dfs,
    mixture_arr= mixture_arr,
    aAS= aAS,
    pot=mwpot,
    kinematic_selection_mask_kwargs= {},
    hf_kwargs= _hf_kwargs_mock,
    ro= ro, vo= vo, zo= zo,
)
print(hf.version)
print(hf.n_star)
hf.get_results()

In [ ]:
figs = make_plots(hf,make_arr,corner_kwargs=corner_kwargs,
                  masses_kwargs=masses_kwargs,
                  dmod_posterior_kwargs=dmod_posterior_kwargs)

### Now plot a mock with disk contamination

In [ ]:
fit_type = 'mock+disk'
mock_truths = halo_mock_params+[fdisk_targ,]
_hf_kwargs_mock = copy.deepcopy(hf_kwargs_mock)
_hf_kwargs_mock['densfunc'] = pdens.triaxial_single_angle_zvecpa_plusexpdisk
_hf_kwargs_mock['fit_type'] = fit_type
_hf_kwargs_mock['version'] = 'hf_mock_halo_'+halo_mock_number+'_disk_'+disk_mock_number
_hf_kwargs_mock['truths'] = mock_truths
_hf_kwargs_mock['truth_mass'] = halo_mock_mass
_hf_kwargs_mock['truths_normed'] = False
_hf_kwargs_mock['int_r_range'] = r_range_halo

hf = pmass.mock_to_hf(
    mock_path = halo_mock_path, 
    dfs = dfs,
    mixture_arr = mixture_arr,
    aAS = aAS,
    pot=mwpot,
    kinematic_selection_mask_kwargs = {},
    hf_kwargs = _hf_kwargs_mock,
    ro = ro, vo = vo, zo = zo,
    disk_mock_path = disk_mock_path,
    fdisk_calc_kwargs = fdisk_calc_kwargs
    )

In [ ]:
figs = make_plots(hf,make_arr,corner_kwargs=corner_kwargs,
                  masses_kwargs=masses_kwargs,
                  dmod_posterior_kwargs=dmod_posterior_kwargs)

#### Now do a mock with KSF

In [ ]:
selecs = ['eLz','AD','JRLz']

fit_type = 'mock+ksf'
mock_truths = copy.deepcopy(halo_mock_params)
_hf_kwargs_mock = copy.deepcopy(hf_kwargs_mock)
_hf_kwargs_mock['densfunc'] = pdens.triaxial_single_angle_zvecpa
_hf_kwargs_mock['fit_type'] = fit_type
_hf_kwargs_mock['truths'] = mock_truths
_hf_kwargs_mock['truth_mass'] = halo_mock_mass
_hf_kwargs_mock['truths_normed'] = False
_hf_kwargs_mock['int_r_range'] = r_range_halo

for i in range(len(selecs)):
    
    _hf_kwargs_mock['selec'] = selecs[i]
    _hf_kwargs_mock['version'] = 'hf_mock_halo_'+halo_mock_number+'_'+selecs[i]+'_ksf_v1'
    kinematic_selection_mask_kwargs['space'] = selecs[i]
    
    hf = pmass.mock_to_hf(
    mock_path= halo_mock_path,
    dfs= dfs,
    mixture_arr= mixture_arr,
    aAS= aAS,
    pot=mwpot,
    kinematic_selection_mask_kwargs= kinematic_selection_mask_kwargs,
    hf_kwargs= _hf_kwargs_mock,
    ro= ro, vo= vo, zo= zo,
    )
    
    figs = make_plots(hf,make_arr,corner_kwargs=corner_kwargs,
                  masses_kwargs=masses_kwargs,
                  dmod_posterior_kwargs=dmod_posterior_kwargs)